In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 29 kB/s 
     |████████████████████████████████| 198 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=e27b57963d9b0ee11e9155b8946171473057be30136f8096d1fd4b9e6bd5b4a8
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [3]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = spark.read.json("drive/My Drive/spot_data.json")

In [5]:
df.show(10)

+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|acousticness|               age|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|
+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|       0.658|41.794520547945204|       0.602|     156067| 0.552|[classic czech po...|00AeAaSNbe92PRrst...|             0.0|  0|  0.0972|  -6.667|   1|               3|      0.404|               10807|                   80|182.229|    

In [6]:
df.printSchema()

root
 |-- acousticness: double (nullable = true)
 |-- age: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- energy: double (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id_track: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: long (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- popularity_track: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- valence: double (nullable = true)



In [7]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler( 
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df)

In [12]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol = "features",outputCol= "scaledFeatures")
scalerData = scaler.fit(output)
out = scalerData.transform(output)

In [13]:
final_data = out.select("id_track", "scaledFeatures", "label")

In [14]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [15]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="scaledFeatures",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.2920857566427187